# Hyperparameter tuning

## Plain scikit-learn

**Hardware**: r5.8xlarge (32 CPU, 256 GB RAM)

In [2]:
from ml_utils import MLUtils

ml_utils = MLUtils(
    ml_task='tip',
    tool='scikit',
    model='random_forest',
)

# Load data and feature engineering

In [3]:
import numpy as np
import pandas as pd

In [4]:
%%time
tip_train = ml_utils.read_parquet_dir(f'{ml_utils.taxi_path}/data/ml/tip_train')
tip_train.shape

CPU times: user 1min 54s, sys: 1min 8s, total: 3min 3s
Wall time: 9min 15s


(219897929, 10)

In [5]:
tip_train.head()

,id,pickup_taxizone_id,dropoff_taxizone_id,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_minute,pickup_week_hour,passenger_count,tip_fraction
0,c92d7d1d969a4c5a89c7dc93f905881e,263.0,236.0,6,27,6,56,150,1.0,0.300000
1,f6f9b9c710ff4adfb29c8687ee646d0d,142.0,163.0,6,27,10,50,154,5.0,0.166667
2,cadce931261a42a2b26878a5f610e900,74.0,66.0,6,27,10,50,154,1.0,0.000000
3,445763736d5d456697048f87ca10f47f,161.0,142.0,6,27,10,50,154,1.0,0.128205
4,51c777d26f2d4f278e8862d2ce6dbd42,226.0,261.0,6,27,10,50,154,1.0,0.166667


Training with the full data ran out of memory, so let's take a sample to be able to successfully train

In [6]:
sample = tip_train.sample(frac=0.5, replace=False, random_state=42)
sample.shape

(109948964, 10)

# Random forest regressor

We don't need to one-hot encode or scale because we are using tree models. We can parallelize the training across all the cores in the single node using `n_jobs=-1`

In [7]:
from sklearn.ensemble import RandomForestRegressor

features = ml_utils.tip_vars.features
y_col = ml_utils.tip_vars.y_col

rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

In [8]:
%%time
_ = rf.fit(sample[features], sample[y_col])

CPU times: user 20h 27min 17s, sys: 2min 36s, total: 20h 29min 53s
Wall time: 44min 31s


## Save model

In [9]:
ml_utils.write_model(rf)

## Predict on test set

And calculate metrics. Save predictions and metrics to S3.

In [10]:
%%time

amt_test = ml_utils.read_parquet_dir(f'{ml_utils.taxi_path}/data/ml/tip_test')
preds = amt_test[['id', y_col]].copy()
preds.columns = ['id', 'actual']
preds['predicted'] = rf.predict(amt_test[features])

CPU times: user 3min 6s, sys: 29.7 s, total: 3min 35s
Wall time: 1min 22s


In [11]:
preds.head()

,id,actual,predicted
0,8e8109754e3e4cb7879c4e9ee216d58d,0.097087,0.159876
1,a30e7c87866f417ab15dee5617f272a0,0.166667,0.157294
2,1a7a611d0809489d99a5120727e0476a,0.120000,0.155907
3,736e84ca12a640cc858c210bd58f744c,0.089474,0.157294
4,f2c24299d9a34ce986b7a271c5cc80b2,0.000000,0.153769


In [12]:
%%time
ml_utils.write_predictions(preds)

CPU times: user 10.5 s, sys: 2.38 s, total: 12.9 s
Wall time: 1min 4s


In [13]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(preds.actual, preds.predicted, squared=False)
ml_utils.write_metric_df('rmse', rmse)

,ml_task,tool,model,metric,value
0,tip,scikit,random_forest,rmse,0.05179
